# QCEW Employee by Industry 15 Country Region Insights

## Setup

### Packages and AGO login

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### Output/input pathing

In [3]:
# Load the uploaded file
INPUT_DATA_PATH = 'inputData/morpc-qcew-2018-2022-annual-MORPC15_readable.csv'

# Directory where images are saved
output_dir = 'outputData/plots'
os.makedirs(output_dir, exist_ok=True)

### Plot colors/paramters

In [20]:
colors = [
    '#365072',  # Original
    '#34617A',  # Original
    '#0075BF',  # Original
    '#00B2BF',  # Original
    '#2A7F67',  # Original
    '#5AB65F',  # Original
    '#28556A',  # Deepened blue-gray
    '#39807D',  # Muted teal
    '#009FCC',  # Brightened blue
    '#33C1C1',  # Softened aqua
    '#3C9563',  # Rich green-teal
    '#66C47A'   # Vibrant green
]

# Dictionary to store COUNTY to itemID mapping to update insights catalog
county_item_mapping = {}

## Pulling Existing Catalog from AGO

In [43]:
# Define the directory to save the data
data_path = Path('inputData/insightsCatalog')
data_path.mkdir(parents=True, exist_ok=True)

# Access the CSV item by ID
catalog_item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
catalog_path = gis.content.get(csv_item_id)

# Download the CSV file
catalog_path = csv_item.download(save_path=data_path)

print(f"CSV file downloaded successfully: {catalog_path}")

CSV file downloaded successfully: inputData\insightsCatalog\morpc_insights_catalog.xlsx


## Creating plots for 15 counties and region summary

In [22]:
# Load the data from the CSV file
data = pd.read_csv(INPUT_DATA_PATH)

# Filter data to exclude 'Total, all industries'
filtered_data = data[data['industry_code'] != 10]

filtered_data = filtered_data[filtered_data['own_code'] == 10]

filtered_data = filtered_data[filtered_data['agglvl_code'] == 73]

filtered_data['county_name'] = filtered_data['county_name'].replace('15-county region', '15County')

# Iterate over each county
for county in filtered_data['county_name'].unique():
    county_data = filtered_data[filtered_data['county_name'] == county]
    
    # Ensure 'YEAR', 'annual_avg_emplvl_prop', and 'industry_code_name' columns are present
    county_data = county_data[['year', 'annual_avg_emplvl_prop', 'industry_code_name']]
    
    # Aggregate to handle duplicates by taking the mean
    county_data = county_data.groupby(['year', 'industry_code_name'], as_index=False).agg({'annual_avg_emplvl_prop': 'mean'})
    
    # Pivot data to have 'industry_code_name' as columns for plotting
    county_data_pivot = county_data.pivot(index='year', columns='industry_code_name', values='annual_avg_emplvl_prop')
    
    # Normalize data for 100% stacking
    county_data_pivot_percentage = county_data_pivot.div(county_data_pivot.sum(axis=1), axis=0) * 100
    
    # Plot each county's data as a 100% stacked bar chart
    ax = county_data_pivot_percentage.plot(
        kind='bar',
        stacked=True,
        figsize=(16, 9),
        color=colors[:len(county_data_pivot_percentage.columns)],  # Use improved color palette
        title=f"{county} Employment Levels Proportions (2018-2022)"
    )
    
    plt.ylabel("Percentage of Employment Proportion")
    plt.xticks(rotation=30, fontsize=12)
    plt.xlabel("")  # Drop x-axis label
    plt.yticks(fontsize=18)
    plt.legend(title='Industry', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=18)  # Set legend font size to 18px
    plt.tight_layout()
    
    # Save the plot as a PNG file
    plot_path = os.path.join(output_dir, f"{county}_employment_levels_100pct_stacked_bar.png")
    plt.savefig(plot_path)
    plt.close()
    
    # Placeholder ID for demonstration as upload to AGO isn't performed here
    county_item_mapping[county] = f"simulated_item_id_for_{county}"

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_file_path_colored = 'AGO_County_ItemID_Mapping.csv'
mapping_df['TilesetID'] = 7
mapping_df.to_csv(mapping_file_path_colored, index=False)

# Provide the path to the saved mappings and generated plots
output_dir, mapping_file_path_colored

('outputData/plots', 'AGO_County_ItemID_Mapping.csv')

## Upload images to AGO and replace existing 

In [23]:
# Initialize dictionary to store county-item mappings
county_item_mapping = {}

# Loop through each file in the directory and upload it
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "tags": "county, population, race, demographics",
            "description": f"Population proportion over time by race for {county_name} county.",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item


        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Store item ID in the dictionary
            county_item_mapping[county_name] = f"https://morpc.maps.arcgis.com/sharing/rest/content/items/{image_item.id}/data"
            
            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df['TilesetID'] = 7
mapping_df.to_csv(mapping_file_path, index=False)



Deleted existing item with title: 15-county region employment levels 100pct stacked bar


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15-county region_employment_levels_100pct_stacked_bar.png with item ID: 4e22d8d612fe47e9a7c4da548452afbb and set it to public


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15County_employment_levels_100pct_stacked_bar.png with item ID: 996fbfb997d748cebf48aec7e349bcac and set it to public
Deleted existing item with title: Delaware employment levels 100pct stacked bar


...

Uploaded Pickaway_employment_levels_100pct_stacked_bar.png with item ID: 9db1ba2185ac4690b2775de5db24c791 and set it to public
Deleted existing item with title: Ross employment levels 100pct stacked bar


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Ross_employment_levels_100pct_stacked_bar.png with item ID: c50a3807c0734d7180202ab81de2eabc and set it to public
Deleted existing item with title: Union employment levels 100pct stacked bar


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_employment_levels_100pct_stacked_bar.png with item ID: 41045993352f486caa7171d263f11e1d and set it to public


## Updating Insights Catalog in AGO

### Local updates to thumbnail

In [46]:
import pandas as pd

# Load the catalog data
catalog_df = pd.read_excel(catalog_path)

# Load the mapping data
mapping_df = pd.read_csv(mapping_file_path_colored)

# Merge the datasets based on the key pairs ('COUNTY', 'TilesetID') in mapping_df 
# and ('GeographyName', 'TilesetID') in catalog_df
merged_df = catalog_df.merge(
    mapping_df,
    left_on=['GeographyName', 'TilesetID'],
    right_on=['COUNTY', 'TilesetID'],
    how='left'
)

# Update 'ThumbnailURL' with 'itemID' where there is a match
merged_df['ThumbnailURL'] = merged_df['itemID'].combine_first(merged_df['ThumbnailURL'])

# Drop unnecessary columns from the merged dataset
merged_df = merged_df.drop(columns=['COUNTY', 'itemID'])

# Save the updated catalog back to an Excel file
updated_catalog_path = "outputData/morpc_insights_catalog_updated.xlsx"
merged_df.to_excel(updated_catalog_path, index=False)

# Provide the file to the user
print(f"Updated catalog saved at: {updated_catalog_path}")


Updated catalog saved at: outputData/morpc_insights_catalog_updated.xlsx


### Pushing excel  changes to AGO

In [49]:
# Define the item ID of the Excel file in AGO
ITEM_ID = "c396b9d0db724d1f870fa47f1d1dc1ce"

# Path to the updated Excel file
updated_catalog_path = "inputData/morpc_insights_catalog_updated.xlsx"

# Get the existing item
item = gis.content.get(ITEM_ID)

if item:
    print(f"Found item: {item.title}")

    # Update the item with the new file
    item.update(
        item_properties={"title": item.title},  # Keeps the existing metadata
        data=updated_catalog_path  # Replaces the file
    )
    
    print("Successfully updated the Excel file in AGO.")
else:
    print("Item not found. Check the item ID and your permissions.")


Found item: morpc_insights_catalog
Successfully updated the Excel file in AGO.


### Updating Feature Layer